### GPT API를 활용하여 학습을 위한 STS dataset 준비

In [1]:
import pandas as pd

df_makests= pd.read_csv('Dataset/kb_dataset_deidentified_oneline_overview_all.csv')

In [3]:
# 모든 문장들 pair로 묶어서 tuple 형태로 저장
sentence_pair_list = []
for idx, row in df_makests.iterrows():
    sentence1 = df_makests.iloc[idx].overview
    num = idx+1
    while num<len(df_makests):
        sentence2 = df_makests.iloc[num].overview
        sentence_pair_list.append((sentence1,sentence2))
        num +=1
    

In [62]:
import openai
from tqdm import tqdm

sts_template = """
당신은 한국어에 능통한 문서 유사도 평가 AI입니다.
아래 튜플 형식으로 제공된 문장들을 꼼꼼하게 읽은 후 각 문장들의 유사도를 평가해주세요.

아래 나열된 지침을 기반으로 생성해주세요.
1. 유사도는 의미론적인 측면에서 평가되어야 한다.
2. 점수는 1점에서 5점 사이로 부여가 되며 첫번째 소수점까지 세밀하게 평가한다.
3. 모든 평가는 일관성있게 측정되어야 한다.
4. 결과는 아래와 같은 형식으로 제공되어야 한다.

원하는 output 형식:
{order}. {score}

[평가 문장 리스트]
{sentences}
"""

nli_template = """
당신은 한국어 NLI 데이터셋을 만드는 전문 AI입니다.
아래 제공된 문장을 꼼꼼하게 읽은 후 해당 문장을 NLI 데이터셋 형태로 만들어주세요.

[제공된 문장]
{sentences}

아래 나열된 지침을 기반으로 생성해주세요.
1. NLI 데이터셋의 label은 (entailment, neutral, contradiction) triplet으로 구성됩니다.
 - entailment는 제공된 문장과 의미적으로 같은 문장을 의미합니다.
 - neutral은 제공된 문장과 의미적으로 관련 없는 문장을 의미합니다.
 - contradiction은 제공된 문장과 의미적으로 모순된 문장을 의미합니다.
2. hyphothesis는 각 label에 해당하는 대표 문장으로 같은 주제 내에서 최대한 창의적으로 작성되어야 한다.
3. 결과는 아래와 같은 json 형식으로 제공되어야 한다.

원하는 output 형식:
{
"premise": {제공된문장},
"nli" : [
{"hyphothesis": {hyphothesis},
"label": entailemnt},
{"hyphothesis": {hyphothesis},
"label": neutral},
{"hyphothesis": {hyphothesis},
"label": contradiction}
]
}


올바른 NLI 데이터셋 예시:
{
"premise": "은행의 이벤트에 참여하시려면 어플에서 행운의 상자를 열고 상품을 확인한 후 응모하셔야 합니다.",
"nli" : [
{"hyphothesis": "이벤트에 참여하려면 어플에서 행운 상자를 열어 상품을 확인한 후 응모하셔야 합니다.",
"label": "entailment"},
{"hyphothesis": "이벤트에 참여하시려면 행운 상자를 열고 상품을 확인한 후 응모하지 않으셔도 됩니다.",
"label": "neutral"},
{"hyphothesis": "이벤트에 참여하시려면 어플에서 럭키박스를 열지 않고 응모하셔야 합니다.",
"label": "contradiction"}
]
}
"""


VALID_CHATGPT_MODELS = ['gpt-3.5-turbo', 'gpt-4']


def get_instructions():
    instructions_dict = {
        'sts' : sts_template,
        'nli' : nli_template
    }
    return instructions_dict

def set_openai_key(key):
    openai.api_key = key

class AskQuestions():

    def __init__(self, text, mode, model='gpt-3.5-turbo'):
        self.text = text
        self.model = model
        self.mode = mode
        self.instruction = get_instructions()

    def ask_question(self):
        instruction = self.instruction['sts'] if self.mode == 'sts' else self.instruction['nli']
        chatgpt_messages = prepare_chatgpt_message(
            instruction,
            self.text
        )
        response = call_chatgpt(chatgpt_messages, model=self.model)

        return response

def prepare_chatgpt_message(task_prompt, text):
    input_prompt = task_prompt.replace('{sentences}', text)
    messages = [{"role": "system", "content": input_prompt}]
    return messages

def call_chatgpt(chatgpt_messages, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(model=model, messages=chatgpt_messages, temperature=0, max_tokens=1024)
    reply = response['choices'][0]['message']['content']
    return reply

open_ai_key = 'chatgpt api key'
set_openai_key(open_ai_key)

In [ ]:
# 문장 튜플 셔플 후 DataFrame 형태로 변환
import random

random.shuffle(sentence_pair_list)
sentence_pair_df = pd.DataFrame(sentence_pair_list, columns=['sent1','sent2'])

In [ ]:
# 문장 튜플 10개씩 chatgpt에 입력해주어 유사도 측정 
def make_score_list(df):
    score_list = [float(score[-3:]) for score in df.split('\n')]
    return score_list

sts_df = pd.DataFrame(columns=['sent1','sent2','score'])
error_idx_list = []
for i in tqdm(range(0,len(sentence_pair_df),10)):
    df = sentence_pair_df.iloc[i:i+10]
    
    sent1 = df.sent1.to_list()
    sent2 = df.sent2.to_list()

    example = [f'{idx+1}. {(sent1,sent2)}' for idx, (sent1, sent2) in enumerate(zip(sent1, sent2))]
    example = '\n'.join(example)

    chat_try = AskQuestions(text=example, mode='sts')
    response = chat_try.ask_question()
    score_list = make_score_list(response)

    if len(score_list) != 10:
        error_idx_list.append(i)
        print(f'index {idx} is not append.')
    else:
        new_row = pd.DataFrame({'sent1' : sent1, 'sent2' : sent2, 'score': score_list})
        sts_df = pd.concat([sts_df,new_row])
 

In [10]:
# STS dataset 저장
sts_df = sts_df.reset_index(drop=True)
sts_df.to_csv('Dataset/kb_dataset_sts.csv',index_label=False)

### GPT API를 활용하여 NLI dataset 만들기

In [54]:
df_makenli = pd.read_csv('Dataset/kb_dataset_all_sentences.csv')

In [55]:
df_makenli.columns = ['text']
df_makenli = df_makenli[df_makenli.text.apply(lambda x: len(x)>=30)].reset_index(drop=True)

In [56]:
df_makenli

,text
0,**은행에서는 단 3일 동안 최대 1천만원을 추첨하는 럭키박스 이벤트를 진행하고 있...
1,"이 이벤트는 **은행의 고객에게만 제공되며, 참여하신 분들 중 한 분을 추첨하여 1..."
2,"이벤트 기간은 9월 28일부터 9월 30일까지이며, 당첨자 발표 및 경품 지급은 1..."
3,"경품은 **은행의 입출금이 자유로운 예금 계좌로 지급되며, 발표일로부터 10영업일 ..."
4,이벤트에 참여하시려면 [Service]에서 럭키박스를 열고 상품을 확인한 후 응모하...
...,...
337,"뱅킹서비스, 알림서비스, 생활/편의서비스, **카드만들기 등 다양한 서비스를 제공하..."
338,"또한 대출/외환서비스, 오픈뱅킹서비스 등도 이용할 수 있습니다."
339,스타뱅킹 길라잡이를 통해 더 편리하고 간편한 은행 업무를 경험해보세요.
340,스타뱅킹 길라잡이와 상담하거나 챗봇을 이용하여 문의할 수 있습니다.


In [70]:
import json


nli_json_list =[]
err_idx_list = []

for i in tqdm(range(175, len(df_makenli))):
    df = df_makenli.iloc[i]
    chat_try = AskQuestions(text=df.text, mode='nli')
    response = chat_try.ask_question()
    try:
        res_json = json.loads(response)
        nli_json_list.append(res_json)
    except:
        print(i)
        err_idx_list.append(i)


100%|██████████| 167/167 [20:55<00:00,  7.52s/it]


In [71]:
df = pd.DataFrame(nli_json_list)
df.to_csv('Dataset/kb_dataset_nli3.csv', index_label=False)

In [72]:
df1 = pd.read_csv('Dataset/kb_dataset_nli.csv')
df2 = pd.read_csv('Dataset/kb_dataset_nli2.csv')
df3 = pd.read_csv('Dataset/kb_dataset_nli3.csv')

In [76]:
df_nli_all = pd.concat([df1,df2,df3]).reset_index(drop=True)

In [ ]:
nli_dict_list = []
def return_one(df):
    nli = eval(df.nli)
    for dict in nli:
        nli_dict = dict
        nli_dict['premise'] = df.premise
        nli_dict_list.append(nli_dict)
df_nli_all.apply(return_one, axis=1)

In [116]:
nli_df = pd.DataFrame('Dataset/kb_dataset_nli.csv')
nli_df.to_csv('Dataset/kb_dataset_nli.csv', index_label=False)

### STS dataset Back translation 통합

In [ ]:
import pandas as pd

df = pd.read_csv('Dataset/sts_augmented.csv')

In [ ]:
original_df = pd.concat([df['sent1'], df['sent2'], df['score']],axis=1)
english_backtranslation_df = pd.concat([df['Augmented_sent1'], df['Augmented_sent2'], df['score']],axis=1)
chinese_backtranslation_df = pd.concat([df['Augmented_sent3'], df['Augmented_sent4'], df['score']],axis=1)

english_backtranslation_df.columns = ['sent1','sent2','score']
chinese_backtranslation_df.columns = ['sent1','sent2','score']

In [ ]:
all_backtranslation_df = pd.concat([original_df, english_backtranslation_df, chinese_backtranslation_df]).reset_index(drop=True)
all_backtranslation_df.to_csv('Dataset/kb_dataset_sts_all.csv')